---
# Grant Jackson - Homework 5
---

### <font color='darkred'> HW5
    
* Use the dataset, "Hitters.csv", posted on BB to explain/predict a baseball player’s salary <u> using a subset of covariates in the dataset </u>.
    
* In order to select the best set of covariates, do the following:
    - LASSO Estimations with CV, AIC and BIC
    - Run regular OLS Regression with non-zero covariates indicated by each CV, AIC, BIC (e.g. Regular OLS Regression excluding covariates dropped (are 0) by LASSO CV, Regular OLS Regression excluding covariates dropped (are 0) by LASSO AIC,.....) 
    - Produce tables or figures or both to summarize your results (use summarycol)
    
* For this exercise, you need to take care of missing values and generate dummies for some variables


In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, LassoLarsIC
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

# Load the data
import os
os.chdir('C:\\Users\gmoor\Documents\Economic Analytics 1\Data')
hitters_data = pd.read_csv('Hitters.csv')

hitters_data.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [2]:
# Display information about the dataset
print("Dataset Info:")
print(hitters_data.info())
print("\nDataset Description:")
print(hitters_data.describe())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  322 non-null    object 
 1   AtBat       322 non-null    int64  
 2   Hits        322 non-null    int64  
 3   HmRun       322 non-null    int64  
 4   Runs        322 non-null    int64  
 5   RBI         322 non-null    int64  
 6   Walks       322 non-null    int64  
 7   Years       322 non-null    int64  
 8   CAtBat      322 non-null    int64  
 9   CHits       322 non-null    int64  
 10  CHmRun      322 non-null    int64  
 11  CRuns       322 non-null    int64  
 12  CRBI        322 non-null    int64  
 13  CWalks      322 non-null    int64  
 14  League      322 non-null    object 
 15  Division    322 non-null    object 
 16  PutOuts     322 non-null    int64  
 17  Assists     322 non-null    int64  
 18  Errors      322 non-null    int64  
 19  Salary      263

In [3]:
# Handle missing values by dropping rows with missing 'Salary' values
hitters_data_clean = hitters_data.dropna(subset=['Salary'])

# Generate dummy variables for the categorical columns 'League', 'Division', and 'NewLeague'
hitters_data_clean = pd.get_dummies(hitters_data_clean, columns=['League', 'Division', 'NewLeague'], drop_first=True)

hitters_data_clean.head()

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League_N,Division_W,NewLeague_N
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,632,43,10,475.0,True,True,True
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,880,82,14,480.0,False,True,False
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,200,11,3,500.0,True,False,True
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,805,40,4,91.5,True,False,True
5,-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,...,501,336,194,282,421,25,750.0,False,True,False


In [4]:
# Prepare the data for LASSO: define X and y
X = hitters_data_clean.drop(columns=['Unnamed: 0', 'Salary'])
y = hitters_data_clean['Salary']

# Convert all columns to numeric, replacing any non-numeric values with NaN
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Drop any rows with NaN values after conversion
X = X.dropna()
y = y[X.index]

print("\nShape of X after cleaning:", X.shape)
print("Shape of y after cleaning:", y.shape)

# Check for any remaining non-numeric data
print("\nData types in X:")
print(X.dtypes)

# Ensure all data is float64
X = X.astype(float)
y = y.astype(float)


Shape of X after cleaning: (263, 19)
Shape of y after cleaning: (263,)

Data types in X:
AtBat          int64
Hits           int64
HmRun          int64
Runs           int64
RBI            int64
Walks          int64
Years          int64
CAtBat         int64
CHits          int64
CHmRun         int64
CRuns          int64
CRBI           int64
CWalks         int64
PutOuts        int64
Assists        int64
Errors         int64
League_N        bool
Division_W      bool
NewLeague_N     bool
dtype: object


In [5]:
# Standardize the predictors
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# Perform LASSO with cross-validation
lasso_cv = LassoCV(cv=5, random_state=0).fit(X_scaled, y)
lasso_cv_coefs = lasso_cv.coef_

lasso_cv_coefs

array([-226.83018915,  254.82705761,    0.        ,   -0.        ,
          0.        ,  102.1450165 ,  -44.5942132 ,   -0.        ,
          0.        ,   43.36306176,  218.02949977,  123.42065271,
       -138.61749903,   76.10383464,   24.74415993,  -13.25243902,
         16.04102919,  -59.54521229,   -0.        ])

In [7]:
# Perform LASSO using AIC and BIC
lasso_aic = LassoLarsIC(criterion='aic').fit(X_scaled, y)
lasso_bic = LassoLarsIC(criterion='bic').fit(X_scaled, y)

lasso_aic_coefs = lasso_aic.coef_
lasso_bic_coefs = lasso_bic.coef_

print(lasso_aic_coefs)
print(lasso_bic_coefs)

[-243.86820725  266.40127753    0.            0.            0.
  106.60985958  -47.53200842    0.            0.           47.51250671
  230.64749689  121.58874342 -151.07487186   76.98509399   27.89478629
  -14.45413345   16.40289964  -59.57562111    0.        ]
[  0.          83.79144232   0.           0.           0.
  47.96218008   0.           0.           0.           0.
  67.30564773 133.75587469   0.          61.11779896   0.
   0.           0.         -51.06531782   0.        ]


In [12]:
# Function to run OLS regression and display the summary
def run_ols(X, y):
    X = sm.add_constant(X)  # Add a constant
    ols_model = sm.OLS(y, X).fit()
    return ols_model

# Get selected columns for each method
cv_selected_columns = X.columns[np.abs(lasso_cv_coefs) > 1e-5]
aic_selected_columns = X.columns[np.abs(lasso_aic_coefs) > 1e-5]
bic_selected_columns = X.columns[np.abs(lasso_bic_coefs) > 1e-5]

# Run OLS regression for each set of selected covariates
print("\nRunning OLS regressions...")
ols_cv_results = run_ols(X[cv_selected_columns], y)
ols_aic_results = run_ols(X[aic_selected_columns], y)
ols_bic_results = run_ols(X[bic_selected_columns], y)

# Print the results
print("\nCV Selected Variables:")
print(cv_selected_columns.tolist())
print("\nAIC Selected Variables:")
print(aic_selected_columns.tolist())
print("\nBIC Selected Variables:")
print(bic_selected_columns.tolist())

print("\nOLS Results with CV-selected variables:")
print(ols_cv_results.summary())
print("\nOLS Results with AIC-selected variables:")
print(ols_aic_results.summary())
print("\nOLS Results with BIC-selected variables:")
print(ols_bic_results.summary())


Running OLS regressions...

CV Selected Variables:
['AtBat', 'Hits', 'Walks', 'Years', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists', 'Errors', 'League_N', 'Division_W']

AIC Selected Variables:
['AtBat', 'Hits', 'Walks', 'Years', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists', 'Errors', 'League_N', 'Division_W']

BIC Selected Variables:
['Hits', 'Walks', 'CRuns', 'CRBI', 'PutOuts', 'Division_W']

OLS Results with CV-selected variables:
                            OLS Regression Results                            
Dep. Variable:                 Salary   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.516
Method:                 Least Squares   F-statistic:                     22.45
Date:                Wed, 09 Oct 2024   Prob (F-statistic):           5.30e-35
Time:                        13:56:15   Log-Likelihood:                -1878.1
No. Observations:                 263   AIC:                 

In [13]:
# Create a summary table using summary_col
models = [ols_cv_results, ols_aic_results, ols_bic_results]
model_names = ['CV', 'AIC', 'BIC']
info_dict = {
    'R-squared': lambda x: f"{x.rsquared:.3f}",
    'Adj. R-squared': lambda x: f"{x.rsquared_adj:.3f}",
    'No. observations': lambda x: f"{int(x.nobs)}",
    'F-statistic': lambda x: f"{x.fvalue:.3f}",
    'Prob (F-statistic)': lambda x: f"{x.f_pvalue:.3f}",
}

summary_table = summary_col(models, 
                            model_names=model_names,
                            stars=True, 
                            float_format='%0.4f',
                            info_dict=info_dict)

# Print the summary table
print("\nSummary Table of OLS Regressions:")
print(summary_table)


Summary Table of OLS Regressions:

                        CV          AIC          BIC     
---------------------------------------------------------
Assists            0.3148       0.3148                   
                   (0.2046)     (0.2046)                 
AtBat              -2.3080***   -2.3080***               
                   (0.5624)     (0.5624)                 
CHmRun             0.8363       0.8363                   
                   (0.8471)     (0.8471)                 
CRBI               0.3573       0.3573       0.4398**    
                   (0.3625)     (0.3625)     (0.1943)    
CRuns              0.9092***    0.9092***    0.2193      
                   (0.2766)     (0.2766)     (0.1915)    
CWalks             -0.8392***   -0.8392***               
                   (0.2721)     (0.2721)                 
Division_W         -119.6740*** -119.6740*** -134.0191***
                   (39.3248)    (39.3248)    (40.5732)   
Errors             -3.2322      -3.2